In [ ]:
import sqlite3
import pandas as pd
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["lang_patient_db"]

# Load the Hypertension collection into a DataFrame
hypertension = pd.DataFrame(list(db["hyper"].find()))

# Verify data integrity
if len(hypertension) < 2:
    print(f"Error: Expected more records, but found {len(hypertension)}. Ensure the database is correctly populated.")
else:
    print(f"Total records loaded: {len(hypertension)}")

# Preprocessing: Drop MongoDB default '_id' and handle missing data
if '_id' in hypertension.columns:
    hypertension.drop('_id', axis=1, inplace=True)
hypertension.fillna("Unknown", inplace=True)

# Create SQLite Database and Table
conn = sqlite3.connect("hypertension.db")
hypertension.to_sql("hypertension", conn, if_exists="replace", index=False)

# ------------------ SQL Queries with Descriptive Outputs ------------------

# 1. Hypertension Patient Demographics
print("\n--- Research Question 1: Hypertension Patient Demographics ---")
query_1 = """
SELECT COUNT(*) AS total_patients,
       (SELECT sex FROM hypertension GROUP BY sex ORDER BY COUNT(*) DESC LIMIT 1) AS most_diagnosed_gender,
       (SELECT age_group FROM (
           SELECT CASE 
                    WHEN age <= 18 THEN '0-18'
                    WHEN age BETWEEN 19 AND 35 THEN '19-35'
                    WHEN age BETWEEN 36 AND 50 THEN '36-50'
                    WHEN age BETWEEN 51 AND 65 THEN '51-65'
                    WHEN age BETWEEN 66 AND 85 THEN '66-85'
                    ELSE '85+'
                 END AS age_group, COUNT(*) AS count
           FROM hypertension
           GROUP BY age_group
           ORDER BY count DESC LIMIT 1
       )) AS most_diagnosed_age_group,
       (SELECT race FROM hypertension GROUP BY race ORDER BY COUNT(*) DESC LIMIT 1) AS most_diagnosed_race,
       (SELECT ethnicity FROM hypertension GROUP BY ethnicity ORDER BY COUNT(*) DESC LIMIT 1) AS most_diagnosed_ethnicity
"""
result_1 = pd.read_sql_query(query_1, conn).iloc[0]
print(f"Total number of hypertensive patients: {result_1['total_patients']}")
print(f"Gender with the most diagnoses: {result_1['most_diagnosed_gender']}")
print(f"Age group with the most diagnoses: {result_1['most_diagnosed_age_group']}")
print(f"Race with the most diagnoses: {result_1['most_diagnosed_race']}")
print(f"Ethnicity with the most diagnoses: {result_1['most_diagnosed_ethnicity']}")

# 2. Medication Adherence
print("\n--- Research Question 2: Medication Adherence ---")
query_2 = """
SELECT 
    medication_name AS most_common_prescription,
    COUNT(*) AS prescription_count,
    (SELECT COUNT(*) FROM hypertension WHERE lastdose <= 7) AS adherent_patients,
    (SELECT COUNT(*) FROM hypertension WHERE lastdose > 7 OR lastdose IS NULL) AS non_adherent_patients,
    ROUND(100.0 * 
        (SELECT COUNT(*) FROM hypertension WHERE lastdose <= 7) / 
        (SELECT COUNT(*) FROM hypertension), 2) AS adherence_rate
FROM hypertension
GROUP BY medication_name
ORDER BY prescription_count DESC
LIMIT 1
"""
result_2 = pd.read_sql_query(query_2, conn).iloc[0]
print(f"Most common prescription: {result_2['most_common_prescription']} with {result_2['prescription_count']} prescriptions.")
print(f"Number of adherent patients: {result_2['adherent_patients']}")
print(f"Adherence rate: {result_2['adherence_rate']}%")


# 3. Adherence by Demographics
print("\n--- Research Question 3: Adherence by Demographics ---")
query_3 = """
SELECT sex, AVG(lastdose) AS avg_adherence_time FROM hypertension GROUP BY sex
"""
result_3 = pd.read_sql_query(query_3, conn)
for _, row in result_3.iterrows():
    print(f"Average adherence time for {row['sex']} patients: {row['avg_adherence_time']:.2f} days")

# 4. Mental Health Conditions
print("\n--- Research Question 4: Mental Health Conditions ---")
query_4 = """
SELECT COUNT(*) AS mental_health_patients
FROM hypertension
WHERE dx_name LIKE '%Depression%' OR dx_name LIKE '%Anxiety%'
"""
result_4 = pd.read_sql_query(query_4, conn).iloc[0]
print(f"Number of hypertensive patients with mental health conditions (Depression or Anxiety): {result_4['mental_health_patients']}")

# 5. Complex Regimens and Adherence
print("\n--- Research Question 5: Complex Regimens and Adherence ---")
query_5 = """
SELECT AVG(lastdose) AS avg_adherence_time
FROM hypertension
WHERE dx_name LIKE '%Depression%' OR dx_name LIKE '%Anxiety%'
"""
result_5 = pd.read_sql_query(query_5, conn).iloc[0]
print(f"Average adherence time for hypertensive patients with mental health conditions: {result_5['avg_adherence_time']:.2f} days")

# 6. Social History Influence
print("\n--- Research Question 6: Social History Influence ---")

# Group by tobacco and alcohol use, calculate average adherence time
query_6 = """
SELECT 
    tobacco_use, 
    alcohol_use, 
    AVG(lastdose) AS avg_adherence_time
FROM hypertension
WHERE lastdose IS NOT NULL
GROUP BY tobacco_use, alcohol_use
"""
result_6 = pd.read_sql_query(query_6, conn)

# Check for completeness of data
required_groups = [
    {"tobacco_use": "Never", "alcohol_use": "Never"},
    {"tobacco_use": "Current", "alcohol_use": "Current"},
    {"tobacco_use": "Never", "alcohol_use": "Current"},
    {"tobacco_use": "Current", "alcohol_use": "Never"},
]

# Verify that all required groups are present
missing_groups = []
for group in required_groups:
    if not (
        (result_6["tobacco_use"] == group["tobacco_use"])
        & (result_6["alcohol_use"] == group["alcohol_use"])
    ).any():
        missing_groups.append(group)

if missing_groups:
    print("Insufficient data for the following groups:")
    for group in missing_groups:
        print(f"Tobacco use: {group['tobacco_use']}, Alcohol use: {group['alcohol_use']}")
    print("Unable to perform analysis due to incomplete data.")
else:
    # Calculate insights
    non_alcohol_users_adherence = result_6[result_6['alcohol_use'] == 'Never']['avg_adherence_time'].mean()
    alcohol_users_adherence = result_6[result_6['alcohol_use'] == 'Current']['avg_adherence_time'].mean()
    non_tobacco_users_adherence = result_6[result_6['tobacco_use'] == 'Never']['avg_adherence_time'].mean()
    tobacco_users_adherence = result_6[result_6['tobacco_use'] == 'Current']['avg_adherence_time'].mean()

    # Generate Insights
    alcohol_insight = (
        f"Non-alcohol users have higher medication adherence ({non_alcohol_users_adherence:.2f} days) "
        f"than current alcohol users ({alcohol_users_adherence:.2f} days)."
        if non_alcohol_users_adherence > alcohol_users_adherence
        else f"Current alcohol users have higher medication adherence ({alcohol_users_adherence:.2f} days) "
             f"than non-alcohol users ({non_alcohol_users_adherence:.2f} days)."
    )

    tobacco_insight = (
        f"Non-tobacco users have higher medication adherence ({non_tobacco_users_adherence:.2f} days) "
        f"than tobacco users ({tobacco_users_adherence:.2f} days)."
        if non_tobacco_users_adherence > tobacco_users_adherence
        else f"Tobacco users have higher medication adherence ({tobacco_users_adherence:.2f} days) "
             f"than non-tobacco users ({non_tobacco_users_adherence:.2f} days)."
    )

    # Print Final Insights
    print(alcohol_insight)
    print(tobacco_insight)

# Close Connection
conn.close()


Total records loaded: 281850

--- Research Question 1: Hypertension Patient Demographics ---
Total number of hypertensive patients: 1
Gender with the most diagnoses: Female
Age group with the most diagnoses: 66-85
Race with the most diagnoses: White
Ethnicity with the most diagnoses: Not of Spanish/Hispanic Origin

--- Research Question 2: Medication Adherence ---
Most common prescription: FLUTICASONE FUROATE 200 MCG-VILANTEROL 25 MCG/DOSE INHALATION POWDER with 16106 prescriptions.
Number of adherent patients: 13105
Adherence rate: 4.65%

--- Research Question 3: Adherence by Demographics ---
Average adherence time for Female patients: 0.27 days
Average adherence time for Male patients: 0.43 days

--- Research Question 4: Mental Health Conditions ---
Number of hypertensive patients with mental health conditions (Depression or Anxiety): 5336

--- Research Question 5: Complex Regimens and Adherence ---
Average adherence time for hypertensive patients with mental health conditions: 0.48 